In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import tree
from sklearn.model_selection import GridSearchCV


import SweetSpotPrediction as ssp

# 04 Classification Model
one of the better models from step 03 can here be tuned
*Note: Run 01 to build the .csv files needed*

In [2]:
df = pd.read_csv('ClassificationData.csv')

In [3]:
features = [
    'XPos', # propbably dont want these as features
    'YPos', # propbably dont want these as features
    'Random1',
    'Random2'

]

label_cls = 'WellPresent' 

In [4]:
X_train, X_, y_train, y_ = train_test_split(df[features], df[label_cls], test_size=0.33, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_, y_, test_size=0.50, random_state=42)

In [5]:
preprocessing = StandardScaler()
X_train = preprocessing.fit_transform(X_train)
X_test = preprocessing.transform(X_test)
X_val = preprocessing.transform(X_val)
X_train[:5]

C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\preprocessing\data.py:617: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\base.py:462: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.fit(X, **fit_params).transform(X)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\ipykernel_launcher.py:4: DataConversionWarning: Data with input dtype int64, float64 were 

array([[-0.92847669, -1.22474487, -0.2363807 ,  0.93460651],
       [ 1.29986737,  1.22474487,  0.79604151, -1.03547357],
       [ 1.29986737, -1.22474487, -1.13892625, -0.79631674],
       [ 0.18569534,  0.        ,  1.72846258,  0.21265195],
       [-0.92847669,  0.        , -1.00202058,  1.59539641]])

In [6]:
from sklearn.ensemble import AdaBoostClassifier

DTC = tree.DecisionTreeClassifier(random_state = 11, max_features = "auto", class_weight = "balanced",max_depth = None)
clf = AdaBoostClassifier(base_estimator = DTC)

In [7]:
clf.fit(X_train, y_train)
y_pred_val = clf.predict(X_val)
ssp.ReportMetrics(clf, X_train, X_val, y_train, y_val, y_pred_val)

Weighted accuracy on training set: 1.000
Weighted accuracy on test set: 0.000

Precision: 0.000
Recall: 0.000
F1: 0.000


C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)
C:\Program Files (x86)\Microsoft Visual Studio\Shared\Anaconda3_64\lib\site-packages\sklearn\metrics\classification.py:1145: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 due to no true samples.
  'recall', 'true', average, warn_for)


# Lets optimize hyper parameters

In [8]:
param_grid = {"base_estimator__criterion" : ["gini", "entropy"],
              "base_estimator__splitter" :   ["best", "random"],
              "n_estimators": [1, 2 , 4, 8 ,16, 32,64,128,256]
             }


# run grid search
grid_search_clf = GridSearchCV(clf, param_grid=param_grid, scoring = 'roc_auc', cv=5,iid=False)
grid_search_clf.fit(X_train, y_train)
grid_search_clf.best_params_
clf = grid_search_clf.best_estimator_


ValueError: n_splits=5 cannot be greater than the number of members in each class.

In [ ]:
y_pred_val = clf.predict(X_val)
ssp.ReportMetrics(clf, X_train, X_val, y_train, y_val, y_pred_val)

# Original

In [ ]:
ssp.plot_map(df, label_clf)

# Prediction

In [ ]:
df['prediction'] = clf.predict(preprocessing.transform(df[features]))
ssp.plot_map(df, 'prediction')